In [2]:
import os
import random
import numpy as np
np.random.seed(1337)  # for reproducibility
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers.core import Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import backend as K
from PIL import Image


Using TensorFlow backend.


In [34]:
Image.LOAD_TRUNCATED_IMAGES = True
K.set_image_dim_ordering('th')
#from keras.backend import tf as ktf
DATA_PATH = "C:\\Users\\Meers\\Documents\\Techie\\kaggle\\data\\"
TRAIN_DIR = "train_1\\"
img_rows, img_cols = 28, 28

In [35]:
def get_image(filename):
    image = load_img(DATA_PATH+TRAIN_DIR+filename)
    image = image.resize((img_rows, img_cols), Image.NEAREST)
    imagedata = img_to_array(image)
    return imagedata

In [36]:
def get_train_images():
    paintings = pd.read_csv("data/train_info1.csv")
    train_images = {filename:get_image(filename) for filename in os.listdir(DATA_PATH+TRAIN_DIR)}
    return paintings, train_images

In [37]:
paintings, train_images = get_train_images()

C:\Users\Meers\AppData\Local\Continuum\Anaconda3\lib\site-packages\PIL\Image.py:2274: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [9]:
print(paintings.get_value(paintings[paintings['filename'] == '1177.jpg'].index[0], 'artist'))

b5d68c68efa0917dc276558a88dfee56


In [38]:
import pickle

with open('train_images.pickle', 'wb') as handle:
    pickle.dump(train_images, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

In [26]:
def get_pairs_alt(paintings, train_images):
    x = []
    y = []
    cnt = 0
    previmg = None
    for img in train_images:
        temp = []
        print(previmg,img)
        if cnt == 1:
            temp.append(train_images[previmg])
            x.append(temp)
            y.append(0)
            #print("x", x[-1][0][0])
            print("y", y[-1])
        elif cnt > 1:
            temp.append(train_images[img])
            temp.append(x[cnt-2][0])
            x.append(temp)
            if((paintings.get_value(paintings[paintings['filename'] == img].index[0], 'artist')) == 
               (paintings.get_value(paintings[paintings['filename'] == previmg].index[0], 'artist'))):
                y.append(1)
            else:
                y.append(0)
            #print("x", x[-1][0][0])
            print("y", y[-1])
        cnt = cnt + 1
        previmg = img
    return x, y

In [33]:
train_images['1.jpg']

KeyError: '1.jpg'

In [61]:
paintings.dropna()
x = []
y = []
prevrow = None
cnt = 0
for index, row in paintings.iterrows():
    pair = []
    if(index == 11025):
        break
    if cnt > 0 :
        pair.append(train_images[prevrow['filename']])
        pair.append(train_images[row['filename']])
        if (prevrow['artist'] == row['artist']):
            y.append(1)
            #print(prevrow['filename'], row['filename'])
            #print(prevrow['artist'], row['artist'])
        else:
            y.append(0)
        #print(prevrow['filename'], row['filename'])
        #print(prevrow['artist'], row['artist'])
        #print(y[-1])
        x.append(pair)
    cnt = cnt + 1
    prevrow = row

In [62]:
with open('training_pairs.pickle', 'wb') as handle:
    pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [63]:
print(len(x),len(y))

11024 11024


In [64]:
#print(len(train_images))

#get_image("10.jpg")
x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.33)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [65]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

In [66]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


In [67]:
def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

In [68]:
def create_base_network(input_dim):
    # input image dimensions
    img_colours, img_rows, img_cols = input_dim

    # number of convolutional filters to use
    nb_filters = 32
    # size of pooling area for max pooling
    nb_pool = 2
    # convolution kernel size
    nb_conv = 3
    model = Sequential()

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            input_shape=(img_colours, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    #model.add(Dropout(0.1)) #0.25 #too much dropout and loss -> nan

    model.add(Flatten())

    model.add(Dense(64, input_shape=(input_dim,), activation='relu'))
    #model.add(Dropout(0.05)) #too much dropout and loss -> nan
    model.add(Dense(32, activation='relu'))
    return model

In [74]:
input_dim = (3, img_rows, img_cols)
nb_epoch = 10
base_network = create_base_network(input_dim)

input_a = Input(shape=(3, img_rows, img_cols,))
input_b = Input(shape=(3, img_rows, img_cols,))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(input=[input_a, input_b], output=distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([x_train[:, 0], x_train[:, 1]], y_train,
          validation_data=([x_test[:, 0], x_test[:, 1]], y_test),
          batch_size=128,
          nb_epoch=nb_epoch)
#model.fit_generator(myGenerator(), samples_per_epoch = 60000, nb_epoch = 2, verbose=2, show_accuracy=True, callbacks=[], validation_data=None, class_weight=None, nb_worker=1)


# compute final accuracy on training and test sets
pred = model.predict([x_train[:, 0], x_train[:, 1]])
tr_acc = compute_accuracy(pred, y_train)
pred = model.predict([x_test[:, 0], x_test[:, 1]])
te_acc = compute_accuracy(pred, y_test)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

C:\Users\Meers\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(3, 28, 28..., padding="valid")`
C:\Users\Meers\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
C:\Users\Meers\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
C:\Users\Meers\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7386 samples, validate on 3638 samples
Epoch 1/10
7386/7386 [==============================] - 13s 2ms/step - loss: 8.5142 - val_loss: 0.2646